In [2]:
from IPython.display import Image 
import pandas
import numpy
import matplotlib
import matplotlib.pyplot as plt 
%matplotlib inline
plt.rcParams["figure.figsize"]=[10,10]

In [3]:
numpy.random.seed(42)

In [4]:
import os
os.path

<module 'ntpath' from 'C:\\Users\\xmax1\\AppData\\Local\\Programs\\Python\\Python37\\lib\\ntpath.py'>

In [5]:
#Seleccion de variables
#En esta ocasion vamos a ver las distintas tecnicas de seleccion de variables, que son las tecnicas que nos permiten filtrar las variables que tenemos y elegir aquwllas que realmente estan ayudando al modelo
#¿Por que querriamos eliminar variables? Generalmente, cuanta mas informacion tenemos sobre cada observacion mejor sera la calidad de nuestras predicciones (cuanto mas sepamos, mas facil sera diferenciar unos casos de otros). No obstante, esto no tiene modelos funcinoen peor, por ejemplo debido a errores de medicion. 
#Seleccionar variables tiene varias ventajas. En primer lugar, como hemos dicho, puede mejorar las puntuaciones de los modelos (eliminando parte del ruido en los datos), por otra parte, reducir el numero de variables independientes reduce la complejidad del modelo, reduciendo asi el efecto de sobreajuste del modelo, Ademas, tener variables independientes y midelos mas simples significa que nuestros modelos son mas rapidos de entrenar, y podemps entrenarlos con mas datos en las mismas maquinas

In [6]:
data=pandas.read_csv(r"D:\Estudiar\MLData\boston\Boston.csv")

In [7]:
data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [8]:
print(data.size)

7084


In [9]:
#Variables numericas
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import normalize

In [10]:
data_num_im_nor=pandas.DataFrame(normalize(SimpleImputer(strategy="median").fit_transform(data)),columns=data.columns)

In [11]:
#Variables categoricas
#Usar sklearn.preprocessing.LabelBinarizer para hacer un 1 hot encoding y codificarlas como vectores
#Usar la funcion get_dummies de pandas
#En general la opcion recomendada es la de usar LabelBinarizer ya que esto nos crea un transformador de scikit learn que poddemos usar en pipelines y para transformar nuevas observaciones
#data_cat_dum=pandas.get_dummies(data_cat,drop_first=True)

In [12]:
#Variables ordinales
#Yendo al diccionario de datos (donde se describen todas las variables) hay varias variables que son ordinales (ordinal, basicamente evaluaciones de una condicion de la casa definidas en una escala no numerica pero que tiene cierto orden (de "mejor" a "peor")

#scikit_learn no tienen un codificador ordinal (Se puede usar el label encoder) Tenemos tres opciones:
#1. implementar nuestro propio OrdinalEncoder
#2. Utilizar la implementacion del paquete scikit_learn.contrib.categorical_encoding, que podemos instalar con pip install category_encoders
#dict_var_ord={
#    "calidad":["buena","mala"],
#    "altura":["NA","baja","alta"]
#}

#col=list(dict_var_ord.keys())
#data_ord=data[col]
#for col_ord,valores in dict_var_ord.items():
#    data_ord[col_ord]=[data_ord[col_ord].astype("category").cat.set_categories(valores).cat.codes]

In [13]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

targetN="medv"
trainN=[col for col in data.columns if col!=targetN]
train=data[trainN]
target=data[targetN]

In [14]:
#La funcion cross_validate que es una version mas flexible que cross_val_score. Evaluaremos usando la raiz del error cuadratico medio (RMSE)

def rmse(y_real,y_pred):
    return numpy.sqrt(mean_squared_error(y_real,y_pred))

def rmse_cv(estimator,X,y):
    y_pred=estimator.predict(X)
    return rmse(y,y_pred)

#scorer = {'main': 'accuracy',
#          'custom': make_scorer(score_func=rmse_cv,greater_is_better=False)}

#scorer = make_scorer(rmse_cv, greater_is_better=False)

scorer={'accuracy': make_scorer(rmse_cv,greater_is_better=False),'prec': 'precision'}

res=cross_validate(LinearRegression(),train,target,scoring=rmse_cv,n_jobs=2,cv=10,return_train_score=True)

In [15]:
res["train_score"]

array([4.83355236, 4.78351694, 4.81830623, 4.55759835, 4.61902872,
       4.72902737, 4.82982623, 3.45820725, 4.6461054 , 4.81550033])

In [16]:
def evaluate_model(estimator,X,y):
    res_estimator=cross_validate(LinearRegression(),train,target,scoring=rmse_cv,n_jobs=2,cv=10,return_train_score=True)
    return res_estimator

resultados={}

def see_res():
    res_df=pandas.DataFrame(resultados).T
    res_cols=res_df.columns
    for col in res_df:
        res_df[col]=res_df[col].apply(numpy.mean)
        res_df[str(col)+"_idx"]=res_df[col]/res_df[col].min()
    return res_df

In [17]:
resultados["reg_lineal_sin_seleccion"]=evaluate_model(LinearRegression(),train,target)
resultados["svr_sin_seleccion"]=evaluate_model(SVR(),train,target)
resultados["rf_sin_seleccion"]=evaluate_model(RandomForestRegressor(),train,target)

see_res()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_lineal_sin_seleccion,0.004801,0.004001,5.180846,4.609067,1.017410,1.250250,1.0,1.0
svr_sin_seleccion,0.004802,0.003200,5.180846,4.609067,1.017557,1.000000,1.0,1.0
rf_sin_seleccion,0.004719,0.003819,5.180846,4.609067,1.000000,1.193202,1.0,1.0


In [18]:
#Metodos de filtrado
#Los metodos de filtrado usan metodos estadisticos para seleccionar las variables que proporcionan la mayor cantidad de informacion. Estos metodos se aplican de forma previa a entrenar el modelo (preprocesado), y son completamente independientes de la eleccion del estimador. Generalmente funcionan definiendo una funcion de evaluacion S(xk_i,y_k), evaluando cada variable independientemente para cada observacion respecto a la variable objetivo de dicha observacion, y eligiendo aquellas k variables que mejor funcionan

#Scikit-learn tiene la siguientes funciones de evaluacion:
    #Para regresion: f_regression, mutual_info_regression
    #Para clasificacion: ch12, f_classif, mutual_info_classif

#f_regression y f_classif devuelven estadisticos F (F_values), entrenando un modelo lineal entre las variables independientes y la objetivo en el caso de regresion, y un test ANOVA en el caso de clasificacion.

#mutual_info_regression y mutual_info_classif computan el coeficiente de informacion mutua (MIC) entre las variables independientes y la variable objetivo

#El coeficiente de informacion mutua nos da una medida de la dependencia entre las variables. El MIC entre dos variables es 0 si no hay relacion entre las mismas, y aumenta conforme mas relacion tienen.

#El MIC se define como:

#con P_xy siendo la probabilidad conjunta de X e Y

#El evaluador ch12 calcula el estadistico chi cuadrado y lo convierte a estadisticos F, con el estadistico chi cuadrado definido como 

In [19]:
from sklearn.feature_selection import SelectKBest, f_regression
sel_kbest=SelectKBest(f_regression,k=10)
data_kbest=sel_kbest.fit_transform(train,target)
data_kbest.shape

(506, 10)

In [20]:
col_sel_kbest=train.loc[:,sel_kbest.get_support()].columns
col_sel_kbest

Index(['crim', 'zn', 'indus', 'nox', 'rm', 'age', 'rad', 'tax', 'ptratio',
       'lstat'],
      dtype='object')

In [21]:
#El parametro scores_ del selector nos devuelve los resultados de la funcion de evaluacion

In [22]:
sel_kbest.scores_[:10]

punt_sel_kbest=zip(train.columns,sel_kbest.scores_, sel_kbest.get_support())

eval_kbest=sorted(
    filter(lambda c: c[2], punt_sel_kbest),
    key=lambda c: c[1], reverse=True
)

In [23]:
list(eval_kbest)

[('lstat', 601.6178711099022, True),
 ('rm', 471.84673987638683, True),
 ('ptratio', 175.10554287576002, True),
 ('indus', 153.95488313610974, True),
 ('tax', 141.76135657742415, True),
 ('nox', 112.59148027970087, True),
 ('crim', 89.48611475768125, True),
 ('rad', 85.91427766984091, True),
 ('age', 83.47745921923685, True),
 ('zn', 75.25764229895405, True)]

In [24]:
#Esto nos permite ver la evaluacion que le da f_regression a cada variable

In [25]:
resultados["reg_lineal_sin_seleccion_kbest"]=evaluate_model(LinearRegression(),data_kbest,target)
resultados["svr_sin_seleccion_kbest"]=evaluate_model(SVR(),data_kbest,target)
resultados["rf_sin_seleccion_kbest"]=evaluate_model(RandomForestRegressor(),data_kbest,target)

In [26]:
see_res()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_lineal_sin_seleccion,0.004801,0.004001,5.180846,4.609067,1.017410,1.250250,1.0,1.0
svr_sin_seleccion,0.004802,0.003200,5.180846,4.609067,1.017557,1.000000,1.0,1.0
rf_sin_seleccion,0.004719,0.003819,5.180846,4.609067,1.000000,1.193202,1.0,1.0
reg_lineal_sin_seleccion_kbest,0.007202,0.005602,5.180846,4.609067,1.526171,1.750205,1.0,1.0
svr_sin_seleccion_kbest,0.004795,0.007208,5.180846,4.609067,1.016066,2.252172,1.0,1.0
rf_sin_seleccion_kbest,0.006403,0.003201,5.180846,4.609067,1.356757,1.000089,1.0,1.0


In [27]:
#metodos envolventes (wrapper methods)

#Los metodos envolventes funcionan de forma similar a los metodos de ranking, sin embargo, en lugar de usar una funcion estadistica independiente del modelo para evaluar las variables. estos metodos usan la funcion de evaluacion o el performance de los modelos como input para decidir que variables elegir (es decir, "envuelven" el funcionamiento del estimador).

#Esto significa que los metodos de filtrado se pueden aplicar independientemente de la eleccion del modelo, ya que consiideran los modelos como una caja negra que produce evaluaciones, aunque clarom, diferentes modelos produciran diferentes selecciones de variables

#scikit-learn implementa un metodo envolvente llamado **recursive feature elimination. 
#Funciona seleccionando tadas las variables, entrenando el modelo, usando los coeficientes 'coef_' o la importancia de las variables 'feature_importances' en funcion del estimador, y eliminando n variables. Este proceso se repite hasta que se alcanza el numero de variables deseado

In [30]:
from sklearn.feature_selection import RFE
estimate_sel=RandomForestRegressor()
sel_rfe_rf=RFE(estimate_sel,n_features_to_select=10)
data_rfe_rf=sel_rfe_rf.fit_transform(train,target)

In [33]:
eval_rfe_rf=sorted(
    filter(lambda c: c[2],
        zip(
            train.columns,
            sel_rfe_rf.ranking_,
            sel_rfe_rf.get_support()
        )
    ), key=lambda c: c[1],reverse=True
)

In [34]:
resultados["reg_lineal_sin_rfe_rf"]=evaluate_model(LinearRegression(),data_rfe_rf,target)
resultados["svr_sin_rfe_rf"]=evaluate_model(SVR(),data_rfe_rf,target)
resultados["rf_sin_rfe_rf"]=evaluate_model(RandomForestRegressor(),data_rfe_rf,target)

In [35]:
see_res()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_lineal_sin_seleccion,0.004801,0.004001,5.180846,4.609067,1.181001,1.250250,1.0,1.0
svr_sin_seleccion,0.004802,0.003200,5.180846,4.609067,1.181171,1.000000,1.0,1.0
rf_sin_seleccion,0.004719,0.003819,5.180846,4.609067,1.160791,1.193202,1.0,1.0
reg_lineal_sin_seleccion_kbest,0.004065,0.004810,5.180846,4.609067,1.000000,1.502943,1.0,1.0
svr_sin_seleccion_kbest,0.004814,0.005206,5.180846,4.609067,1.184262,1.626529,1.0,1.0
rf_sin_seleccion_kbest,0.004178,0.003250,5.180846,4.609067,1.027681,1.015435,1.0,1.0


In [37]:
from sklearn.feature_selection import RFECV
estimate_sel=RandomForestRegressor()
sel_rfecv_rf=RFE(estimate_sel)
data_rfecv_rf=sel_rfecv_rf.fit_transform(train,target)

In [38]:
resultados["reg_lineal_sin_rfecv_rf"]=evaluate_model(LinearRegression(),data_rfe_rf,target)
resultados["svr_sin_rfecv_rf"]=evaluate_model(SVR(),data_rfe_rf,target)
resultados["rf_sin_rfecv_rf"]=evaluate_model(RandomForestRegressor(),data_rfe_rf,target)

In [39]:
see_res()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_lineal_sin_seleccion,0.004801,0.004001,5.180846,4.609067,1.181001,2.393414,1.0,1.0
svr_sin_seleccion,0.004802,0.003200,5.180846,4.609067,1.181171,1.914349,1.0,1.0
rf_sin_seleccion,0.004719,0.003819,5.180846,4.609067,1.160791,2.284205,1.0,1.0
reg_lineal_sin_seleccion_kbest,0.004065,0.004810,5.180846,4.609067,1.000000,2.877157,1.0,1.0
svr_sin_seleccion_kbest,0.004814,0.005206,5.180846,4.609067,1.184262,3.113745,1.0,1.0
rf_sin_seleccion_kbest,0.004178,0.003250,5.180846,4.609067,1.027681,1.943898,1.0,1.0
reg_lineal_sin_rfecv_rf,0.005942,0.001672,5.180846,4.609067,1.461660,1.000000,1.0,1.0
svr_sin_rfecv_rf,0.004366,0.002663,5.180846,4.609067,1.074012,1.593066,1.0,1.0
rf_sin_rfecv_rf,0.004133,0.004053,5.180846,4.609067,1.016603,2.424389,1.0,1.0


In [ ]:
#Metodos embebidos (Embedded Methods)
#Son metodos de seleccion de variables que estan integrados dentro del proceso de entrenamiento de modelos, y son especificos para cada modelo

#El ejemplo clasico de metodo embebido de seleccion de variables son los procesos de regularizacion en regresion lineal. Por ejemplo el metodo L1 tiene a convertir los coeficientes de la variables que no funcionan a 0, por lo tanto eliminandolas